In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
path = '/content/gdrive/My Drive/Colab Notebooks/Thesis/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.special import inv_boxcox
from sklearn import preprocessing
from sklearn.utils import shuffle

%matplotlib inline

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular import *
from fastai.tabular.all import *
from fastai.tabular.data import *
from fastai.tabular.learner import *
from fastai.metrics import *

In [ ]:
def predict_model(df, xs, joblib_file, name):  
    joblib_model = joblib.load(path + joblib_file)
    preds = joblib_model.predict(xs)
    df = df.copy()
    df[name] = preds
    return df

In [ ]:
in_put = path + 'data2020.csv'
out_put = in_put
mon = '06'

In [ ]:
alist = ['area', 'night', 'road_len', 'city_pop', 'cen_dist', 'X', 'Y', 'land_cover','city_id']
name1, name2 = 'population_cat', 'population_preds'
joblib_file1, joblib_file2 = 'classifier.pkl', 'DL.pkl'
procs = [Categorify,]
cat_names = alist[-2:]
cont_names = alist[1:-2]

In [ ]:
df_ori = pd.read_csv(in_put)
df_ori['night'] = df_ori['night'+mon]
df_ori.head()

,fid,country,fua_name,road_len,area,population,city_pop,cen_dist,night01,night06,night05,night04,night03,night02,X,Y,pop01,pop02,pop03,pop04,pop05,pop06,land_cover,city_id,delta2,delta3,delta4,delta5,delta6,night
0,1,PT,Lisboa,0.000000,669832.232177,0.0,2815535,38.606784,0.8300,0.9275,0.7325,1.0125,1.1225,0.8650,-9.505698,38.784757,0.000000,0.000000,0.0,0.000000,0.0,0.000000,11,21,0.000000,0.000000,0.000000,0.000000,0.000000,0.9275
1,2,PT,Lisboa,0.000000,669755.214578,0.0,2815535,37.800579,0.8200,0.8450,0.8525,1.1550,1.0700,0.8650,-9.497365,38.793090,0.000000,0.000000,0.0,0.000000,0.0,0.000000,11,21,0.000000,0.000000,0.000000,0.000000,0.000000,0.8450
2,3,PT,Lisboa,1.027019,669832.232177,0.0,2815535,37.685759,1.1675,1.4050,1.5025,1.3425,1.5100,1.2225,-9.497365,38.784757,0.994022,1.093224,0.0,1.325878,0.0,5.348628,11,21,0.216821,-4.623647,5.099267,-5.099267,5.348628,1.4050
3,4,PT,Lisboa,0.083511,669909.235391,0.0,2815535,37.592849,1.1925,1.3450,1.4600,2.0350,1.6550,1.3800,-9.497365,38.776424,0.455709,0.000000,0.0,0.000000,0.0,1.249402,11,21,-1.018882,0.000000,0.000000,0.000000,1.249402,1.3450
4,5,PT,Lisboa,0.000000,669986.224222,0.0,2815535,37.522009,0.8300,0.9450,0.8325,1.2600,1.1550,0.8725,-9.497365,38.768090,0.000000,0.000000,0.0,0.000000,0.0,0.000000,11,21,0.000000,0.000000,0.000000,0.000000,0.000000,0.9450


In [ ]:
df_ori.describe()

,fid,road_len,area,population,city_pop,cen_dist,night01,night06,night05,night04,night03,night02,X,Y,pop01,pop02,pop03,pop04,pop05,pop06,land_cover,city_id,delta2,delta3,delta4,delta5,delta6,night
count,260534.000000,260534.000000,260534.000000,260534.000000,2.605340e+05,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,256219.000000,260386.000000,240507.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000
mean,130267.500000,2.618643,553544.642024,573.218713,4.394432e+06,48.840822,4.644294,5.255424,5.193526,5.253653,5.165468,5.315554,9.948661,49.823713,491.481889,522.360391,527.914116,530.052374,554.755166,447.490778,6.626156,11.373537,21.840706,-0.280784,7.488709,-10.584839,-18.674602,5.255424
std,75209.831854,3.395864,56618.677496,2237.603599,3.148661e+06,26.506507,14.096843,11.833709,11.813289,12.606266,13.708143,18.847539,8.835994,5.054657,1846.483277,1953.447752,1940.070025,1962.336037,1963.734436,1618.507675,2.141825,5.586927,121.709814,171.356549,161.609543,270.209529,292.011629,11.833709
min,1.000000,0.000000,428350.687197,0.000000,1.207469e+06,0.088187,0.032500,0.000000,0.000000,-0.102500,-0.050000,0.137500,-9.505698,38.409757,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-3461.670971,-15286.779841,-2698.539731,-16071.703089,-15846.081697,0.000000
25%,65134.250000,0.079500,522396.564847,0.000000,2.120416e+06,29.361106,0.365000,1.052500,0.882500,0.427500,0.452500,0.522500,2.452635,47.718090,0.548192,2.544199,1.819989,1.250209,5.052087,2.176052,7.000000,7.000000,0.000000,-2.688741,-2.683787,0.000000,-0.381519,1.052500
50%,130267.500000,1.553339,545090.648482,10.003517,2.939714e+06,44.830977,0.807500,1.765000,1.307500,0.885000,0.965000,0.995000,12.369302,50.759757,12.282942,17.370163,16.078578,13.806503,28.726626,20.303976,7.000000,12.000000,1.307667,0.000000,0.000000,1.959839,0.066443,1.765000
75%,195400.750000,3.376450,579327.075616,125.796494,4.814757e+06,63.796209,2.910000,3.502500,3.360000,3.315000,3.377500,3.280000,16.669302,52.701424,85.540257,99.463143,98.865243,93.449087,127.973952,99.582193,8.000000,17.000000,8.214450,1.019633,1.178621,13.856539,4.860333,3.502500
max,260534.000000,40.156530,673283.116298,58890.018027,1.166596e+07,166.826791,1885.490022,1044.810000,769.134992,768.860018,1427.142518,3574.597595,26.460969,60.259757,45266.837422,45921.397908,48326.964055,47153.420554,47206.568016,34797.079891,11.000000,22.000000,3736.603399,3598.217604,15343.688614,2099.475231,1944.953644,1044.810000


In [ ]:
df = df_ori[alist]
to = TabularPandas(df[alist[1:]], procs=procs, cat_names=cat_names, cont_names=cont_names)
df = predict_model(df, xs=to.xs, joblib_file=joblib_file1, name=name1)
df.shape

(260534, 10)

In [ ]:
print('Step I: Zero-Populated Areas Detected!')
df[df[name1]==1].shape

Step I: Zero-Populated Areas Detected!


(217071, 10)

In [ ]:
# Based on city_id, to normalize the longitude and latitude
blist = ['X','Y']
for col in blist:
    df[col+'_max'] = df.groupby(['city_id'])[col].transform(max)
    df[col+'_min'] = df.groupby(['city_id'])[col].transform(min)
    df[col] = ((df[col] - df[col+'_min']) / (df[col+'_max'] - df[col+'_min']))
    df[col] 

In [ ]:
# MinMaxScaler
scaler = preprocessing.MinMaxScaler()
blist = ['city_pop',]
for name in blist:
    df[name] = scaler.fit_transform(df[name].values.reshape(-1, 1))

In [ ]:
box_cox, mean, std = \
({'cen_dist': 0.4544457663131222,
  'night': -0.22553801556625228,
  'population': 0.02837279324792106,
  'road_len': 0.2948282226915623},
 {'X': 0.48962825046566666,
  'Y': 0.4984488592873198,
  'cen_dist': 9.762960323420865,
  'night': 0.2197375900273375,
  'population': 3.9487919852473183,
  'road_len': 0.7171207244705375},
 {'X': 0.22244316071968143,
  'Y': 0.21778393142042998,
  'cen_dist': 3.2444866504064005,
  'night': 1.3003199440371558,
  'population': 3.2221005079579674,
  'road_len': 1.8754672371416614})

In [ ]:
def standardlization(df, name, box_cox, std, mean, box_true=True):
    if box_true:
        df[name] = stats.boxcox(df[name], lmbda=box_cox[name], alpha=None)
    df[name] = (df[name] - mean[name])/ std[name]
    return df

def inverse_standardlization(df, name, name0, box_cox, std, mean):
    df[name] = df[name] * std[name0] + mean[name0]
    df[name] = inv_boxcox(df[name], box_cox[name0])
    return df

In [ ]:
df['night'] = df['night'].apply(lambda x: max(x, 0.000001))
df['road_len'] = df['road_len'] + 0.000001

In [ ]:
blist = ['road_len', 'night', 'cen_dist']
for name in blist:
    df = standardlization(df, name, box_cox, std, mean)
    
blist = ['X', 'Y']
for name in blist:
    df = standardlization(df, name, box_cox, std, mean, box_true=False)

In [ ]:
df.describe()

,area,night,road_len,city_pop,cen_dist,X,Y,land_cover,city_id,population_cat,X_max,X_min,Y_max,Y_min
count,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000,260534.000000
mean,553544.642024,-7.484082,-0.380922,0.304725,0.129972,0.025652,-0.034392,6.626156,11.373537,0.833177,10.845051,9.076603,50.387242,49.287537
std,56618.677496,22.886650,1.170256,0.301063,1.000185,1.031663,1.032044,2.141825,5.586927,0.372818,8.923043,8.769045,5.137996,4.994238
min,428350.687197,-73.664805,-2.160095,0.000000,-3.462349,-2.201139,-2.288731,1.000000,1.000000,0.000000,-8.489031,-9.505698,39.251424,38.409757
25%,522396.564847,-0.129863,-1.333605,0.087292,-0.536703,-0.780138,-0.858527,7.000000,7.000000,1.000000,3.560969,1.319302,48.026424,47.151424
50%,545090.648482,0.241108,-0.131616,0.165630,0.131466,0.032310,-0.050756,7.000000,12.000000,1.000000,13.327635,11.819302,51.009757,50.493090
75%,579327.075616,0.670709,0.398086,0.344915,0.795526,0.834060,0.762135,8.000000,17.000000,1.000000,17.160969,15.569302,53.251424,51.834757
max,673283.116298,2.529899,3.181342,1.000000,3.251275,2.294392,2.302976,11.000000,22.000000,1.000000,26.460969,25.860969,60.259757,58.734757


In [ ]:
# df = pd.read_csv(path+'data2_sf.csv')
# percent, length = 0.8, len(df.axes[0])
# stop = int(percent * length)
# valid_idx = list(range(stop,length))
# df = df.iloc[valid_idx]
# df.describe()

In [ ]:
learn = load_learner(path+joblib_file2)
test = df[alist[1:]]
dl = learn.dls.test_dl(test)
dl.show()

,land_cover,city_id,night,road_len,city_pop,cen_dist,X,Y
0,11,21,-0.227361,-2.160095,0.153757,-0.119317,-2.201139,-0.242921
1,11,21,-0.301000,-2.160095,0.153757,-0.153372,-2.164290,-0.197459
2,11,21,0.082739,-0.368097,0.153757,-0.158255,-2.164290,-0.242921
3,11,21,0.051500,-1.321075,0.153757,-0.162211,-2.164290,-0.288384
4,11,21,-0.212771,-2.160095,0.153757,-0.165232,-2.164290,-0.333846
5,11,21,0.012822,-2.160095,0.153757,-0.179663,-2.127441,-0.106534
6,11,21,0.059468,-2.160095,0.153757,-0.186611,-2.127441,-0.151996
7,9,21,0.071220,-2.160095,0.153757,-0.192630,-2.127441,-0.197459
8,9,21,0.281734,-0.314482,0.153757,-0.197702,-2.127441,-0.242921
9,9,21,0.443951,-1.242345,0.153757,-0.201813,-2.127441,-0.288384


In [ ]:
preds, targs = learn.get_preds(dl=dl)

In [ ]:
df[name2] = preds.numpy()
preds

tensor([[-1.7358],
        [-1.8159],
        [-1.1090],
        ...,
        [-0.9672],
        [-0.8228],
        [-0.7900]])

In [ ]:
df = df.drop(['X_max','X_min','Y_max','Y_min'], axis=1)
df.head()

,area,night,road_len,city_pop,cen_dist,X,Y,land_cover,city_id,population_cat,population_preds
0,669832.232177,-0.227361,-2.160095,0.153757,-0.119317,-2.201139,-0.242921,11,21,0,-1.735818
1,669755.214578,-0.301000,-2.160095,0.153757,-0.153372,-2.164290,-0.197459,11,21,0,-1.815873
2,669832.232177,0.082739,-0.368097,0.153757,-0.158255,-2.164290,-0.242921,11,21,1,-1.109012
3,669909.235391,0.051500,-1.321075,0.153757,-0.162211,-2.164290,-0.288384,11,21,1,-1.396234
4,669986.224222,-0.212771,-2.160095,0.153757,-0.165232,-2.164290,-0.333846,11,21,0,-1.709958


In [ ]:
df = inverse_standardlization(df, name2, 'population', box_cox, std, mean)
# data = inverse_standardlization(data, 'population', 'population', box_cox, std, mean)
df[name2] = df[name2] * df[name1]
df[name2] = df[name2].apply(lambda x: max(x,0))

In [ ]:
df_ori['pop'+mon] = df[name2]

In [ ]:
df_ori['delta2'] = df_ori['pop02'] - df_ori['pop01']
df_ori['delta3'] = df_ori['pop03'] - df_ori['pop02']
df_ori['delta4'] = df_ori['pop04'] - df_ori['pop03']
df_ori['delta5'] = df_ori['pop05'] - df_ori['pop04']
df_ori['delta6'] = df_ori['pop06'] - df_ori['pop05']

In [ ]:
df_ori.to_csv(out_put, index=False)
print('Step II: Populated Areas Predicted!')
df_ori.head()

Step II: Populated Areas Predicted!


,fid,country,fua_name,road_len,area,population,city_pop,cen_dist,night01,night06,night05,night04,night03,night02,X,Y,pop01,pop02,pop03,pop04,pop05,pop06,land_cover,city_id,delta2,delta3,delta4,delta5,delta6,night
0,1,PT,Lisboa,0.000000,669832.232177,0.0,2815535,38.606784,0.8300,0.9275,0.7325,1.0125,1.1225,0.8650,-9.505698,38.784757,0.000000,0.000000,0.0,0.000000,0.0,0.000000,11,21,0.000000,0.000000,0.000000,0.000000,0.000000,0.9275
1,2,PT,Lisboa,0.000000,669755.214578,0.0,2815535,37.800579,0.8200,0.8450,0.8525,1.1550,1.0700,0.8650,-9.497365,38.793090,0.000000,0.000000,0.0,0.000000,0.0,0.000000,11,21,0.000000,0.000000,0.000000,0.000000,0.000000,0.8450
2,3,PT,Lisboa,1.027019,669832.232177,0.0,2815535,37.685759,1.1675,1.4050,1.5025,1.3425,1.5100,1.2225,-9.497365,38.784757,0.994022,1.093224,0.0,1.325878,0.0,1.452749,11,21,0.099202,-1.093224,1.325878,-1.325878,1.452749,1.4050
3,4,PT,Lisboa,0.083511,669909.235391,0.0,2815535,37.592849,1.1925,1.3450,1.4600,2.0350,1.6550,1.3800,-9.497365,38.776424,0.455709,0.000000,0.0,0.000000,0.0,0.574446,11,21,-0.455709,0.000000,0.000000,0.000000,0.574446,1.3450
4,5,PT,Lisboa,0.000000,669986.224222,0.0,2815535,37.522009,0.8300,0.9450,0.8325,1.2600,1.1550,0.8725,-9.497365,38.768090,0.000000,0.000000,0.0,0.000000,0.0,0.000000,11,21,0.000000,0.000000,0.000000,0.000000,0.000000,0.9450


In [ ]:
# data = pd.read_csv(path+'data2.csv')
# a = data['population']
# b = df[name2]

# plt.figure(figsize=(4,3),dpi=200)
# plt.hist(a, bins=500, alpha=0.5, label='Population (2012)', density=True)
# plt.hist(b, bins=500, alpha=0.5, label='Population Prediction\n(Validation Set)', density=True)

# plt.xlabel('Data')
# plt.ylabel('Density')
# # plt.xlim(-4, 4)
# plt.rcParams.update({'font.size': 7})
# plt.legend(loc='best')
# plt.grid(linestyle='-.',linewidth=0.3)
# plt.show()

In [ ]:
data['pop_1'] = data['population'] * data['area'] / 1e12
data['pop_2'] = data.groupby(['city_id'])['pop_1'].transform(sum)
df['pop_3'] = df[name2] * df['area'] / 1e12
df['pop_4'] = df.groupby(['city_id'])['pop_3'].transform(sum)

abs = pd.DataFrame({
    'city': df_ori['fua_name'].unique(),
    'pop_2012': data['pop_2'].unique(), 
    'pop_preds': df['pop_4'].unique()
})

In [ ]:
abs['error'] = (abs['pop_preds'] / abs['pop_2012'] - 1) * 100
abs

In [ ]:
abs['error'].mean()+100